In [ ]:
#!pip install --upgrade google-cloud-videointelligence

In [ ]:
from google.cloud import videointelligence
import yaml
import os
from google.protobuf.json_format import MessageToDict
from google.cloud import storage
import json

In [ ]:
#not needed when the notebook owner is the project with permissions
#with open('properties.yaml') as file:
#    properties = yaml.full_load(file)

#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = properties['google_application_credentials']

In [ ]:
# note - this threw an error about Feature not being available on videointelligence
# I reinstalled through pip above and reset kernel, and then it ran. 
# not sure why but try that if you have trouble.

video_client = videointelligence.VideoIntelligenceServiceClient()
features = [videointelligence.Feature.SPEECH_TRANSCRIPTION]

In [ ]:
config = videointelligence.SpeechTranscriptionConfig(
    language_code="en-US", enable_automatic_punctuation=True
)

In [ ]:
path = 'gs://idl-dsos-transcript/video_files/archive.org/download/20230829073017_opioids_xypn0282/xypn0282-0001.mp4'

video_context = videointelligence.VideoContext(speech_transcription_config=config)

operation = video_client.annotate_video(
    request={
        "features": features,
        "input_uri": path,
        "video_context": video_context,
    }
)

video_context = videointelligence.VideoContext(speech_transcription_config=config)


operation = video_client.annotate_video(
    request={
        "features": features,
        "input_uri": path,
        "video_context": video_context,
    }
)

print("\nProcessing video for speech transcription.")

result = operation.result(timeout=6000)

# There is only one annotation_result since only
# one video is processed.
#annotation_results = result.annotation_results[0]
#for speech_transcription in annotation_results.speech_transcriptions:
    # The number of alternatives for each transcription is limited by
    # SpeechTranscriptionConfig.max_alternatives.
    # Each alternative is a different possible transcription
    # and has its own confidence score.
    #for alternative in speech_transcription.alternatives:
        #print("Alternative level information:")

        #print("Transcript: {}".format(alternative.transcript))
        #print("Confidence: {}\n".format(alternative.confidence))

        #print("Word level information:")
        #for word_info in alternative.words:
            #word = word_info.word
            #start_time = word_info.start_time
            #end_time = word_info.end_time
            #print(
                #"\t{}s - {}s: {}".format(
                    #start_time.seconds + start_time.microseconds * 1e-6,
                    #end_time.seconds + end_time.microseconds * 1e-6,
                    #word,
                #)
            #)

In [ ]:
annotation_results = result.annotation_results[0]

In [ ]:
transcripts = []
confidences = []

for speech_transcription in annotation_results.speech_transcriptions:
    # The number of alternatives for each transcription is limited by
    # SpeechTranscriptionConfig.max_alternatives.
    # Each alternative is a different possible transcription
    # and has its own confidence score.
    for alternative in speech_transcription.alternatives:
       
        if alternative.transcript != "":
            transcripts.append(alternative.transcript)
            confidences.append(alternative.confidence)
    
print(''.join(transcripts))

In [ ]:
# without word level information

for speech_transcription in annotation_results.speech_transcriptions:
    # The number of alternatives for each transcription is limited by
    # SpeechTranscriptionConfig.max_alternatives.
    # Each alternative is a different possible transcription
    # and has its own confidence score.
    for alternative in speech_transcription.alternatives:
        print("Alternative level information:")

        print("Transcript: {}".format(alternative.transcript))
        print("Confidence: {}\n".format(alternative.confidence))

In [ ]:
type(annotation_results)

In [ ]:
result_dict = MessageToDict(annotation_results.__class__.pb(annotation_results))
#print(result_dict)

In [ ]:
type(result_dict)

In [ ]:
gcs_destination_uri = "gs://idl-dsos-transcript/json_files/"

In [ ]:
bucket_name = 'idl-dsos-transcript'
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob("json_files/sthh0257-0001.json")
blob.upload_from_string(json.dumps(result_dict))